In [1]:
import numpy as np
import itertools
from skimage import color, io

P = 0.3
Q = 0.0625

LEFT = 0
RIGHT = 1
UP = 2
DOWN = 3

In [2]:
def load_image(path):
    return color.rgb2lab(io.imread(path))


def split_into_squares(image, nrows, ncols):
    return list(itertools.chain.from_iterable(np.hsplit(r, ncols) for r in np.split(image, nrows)))


def calculate_dissimilarity(x_i, x_j, relation):
    nrows, ncols, _ = x_i.shape

    if relation == LEFT:
        return calculate_dissimilarity(x_j, x_i, RIGHT)
    elif relation == RIGHT:
        return np.sum(
            np.power(np.power(np.abs((2 * x_i[:, ncols - 1] - x_i[:, ncols - 2]) - x_j[:, 0]), P) +
                     np.power(np.abs((2 * x_j[:, 0] - x_j[:, 1]) - x_i[:, ncols - 1]), P), Q / P))
    elif relation == UP:
        return calculate_dissimilarity(x_j, x_i, DOWN)
    elif relation == DOWN:
        return np.sum(
            np.power(np.power(np.abs((2 * x_i[nrows - 1] - x_i[nrows - 2]) - x_j[0]), P) +
                     np.power(np.abs((2 * x_j[0] - x_j[1]) - x_i[nrows - 1]), P), Q / P))
    else:
        raise TypeError(f'invalid relation: {relation}')


def build_dissimilarity_matrix(squares):
    dissimilarity_matrix = np.empty((4, len(squares), len(squares)))
    for relation in range(4):
        for i, x_i in enumerate(squares):
            for j, x_j in enumerate(squares):
                if i == j:
                    continue
                dissimilarity_matrix[relation][i][j] = calculate_dissimilarity(x_i, x_j, relation)
    return dissimilarity_matrix


def calculate_compatibility(dissimilarity_matrix, i, j, relation):
    return np.exp(-dissimilarity_matrix[relation][i][j] /
                  np.percentile(np.delete(dissimilarity_matrix[relation][i], i), 25))


def build_compatibility_matrix(dissimilarity_matrix):
    _, order, _ = dissimilarity_matrix.shape
    compatibility_matrix = np.empty((4, order, order))
    for relation in range(4):
        for i in range(order):
            for j in range(order):
                if i == j:
                    continue
                compatibility_matrix[relation][i][j] = calculate_compatibility(dissimilarity_matrix, i, j, relation)
    return compatibility_matrix


def calculate_best_neighbours(compatibility_matrix):
    _, order, _ = compatibility_matrix.shape
    best_neighbours = np.zeros((4, order), dtype=int)
    for relation in range(4):
        for i in range(order):
            best_neighbours[relation][i] = np.argmax(compatibility_matrix[relation][i])
    return best_neighbours


def opposite_relation(relation):
    if relation == 0 or relation == 2:
        return relation + 1
    else:
        return relation - 1

In [71]:
def find_best_estimated_seed(best_neighbours):
    _, order = best_neighbours.shape
    num_best_buddies = np.zeros(order, dtype=int)
    for relation in range(4):
        for i in range(order):
            buddy = best_neighbours[relation][i]
            opposite = opposite_relation(relation)
            if best_neighbours[opposite][buddy] == i:
                num_best_buddies[i] += 1
    return np.argmax(num_best_buddies)


In [4]:
image = load_image('../data/data_train/64/1200.png')
squares = split_into_squares(image, 8, 8)
dissimilarity_matrix = build_dissimilarity_matrix(squares)
compatibility_matrix = build_compatibility_matrix(dissimilarity_matrix)
best_neighbours = calculate_best_neighbours(compatibility_matrix)
print(best_neighbours[RIGHT][0])
print(best_neighbours[LEFT][3])

3
0


In [7]:
print(find_best_estimated_seed(best_neighbours))

[4 4 3 4 4 4 3 4 3 3 4 3 4 4 3 4 4 4 4 4 4 3 3 3 4 4 4 4 4 3 2 4 2 3 4 4 4
 4 4 3 4 3 4 3 4 2 4 3 4 2 4 3 3 4 4 3 2 3 4 4 3 4 4 4]
0


In [17]:
arr = np.arange(9).reshape((3, 3))
print(arr)
print(np.roll(arr, 1, 1))

[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[2 0 1]
 [5 3 4]
 [8 6 7]]


In [24]:
def neighbours(puzzle, i, j):
    nrows, ncols = puzzle.shape
    if i > 0:
        yield i -1, j
    if i < nrows - 1:
        yield i + 1, j
    if j > 0:
        yield i, j- 1
    if j < ncols - 1:
        yield i, j+1

In [60]:
def initialise_occupied_neighbours(puzzle):
    nrows, ncols = puzzle.shape
    num_occupied_neighbours = np.zeros((nrows, ncols), dtype=int)
    for i in range(nrows):
        for j in range(ncols):
            if puzzle[i][j] >= 0:
                continue
            for x, y in neighbours(puzzle, i, j):
                if puzzle[x][y] >= 0:
                    num_occupied_neighbours[i][j] += 1
    return num_occupied_neighbours

In [61]:
def find_candidate_slots(num_occupied_neighbours):
    nrows, ncols = num_occupied_neighbours.shape
    most_occupied_neighbours = np.max(num_occupied_neighbours)            
    for i in range(nrows):
        for j in range(ncols):
            if num_occupied_neighbours[i][j] == most_occupied_neighbours:
                yield i, j

In [63]:
puzzle = np.full((3, 3), -1)
puzzle[1][1] = 1
occupied_neighbours = initialise_occupied_neighbours(puzzle)
assert list(find_candidate_slots(occupied_neighbours)) == [(0, 1), (1, 0), (1, 2), (2, 1)]
puzzle[0][1] = 1
puzzle[1][0] = 1
occupied_neighbours = initialise_occupied_neighbours(puzzle)
assert list(find_candidate_slots(occupied_neighbours)) == [(0, 0)]

In [64]:
def best_buddies(best_neighbours, relation, i, j):
    return best_neighbours[relation][i] == j and best_neighbours[opposite_relation(relation)][j] == i

In [65]:
# part fits in slot if it is best buddies with all the occupied neighbours of that slot
def does_part_fit_in_slot(puzzle, best_neighbours, slot, part):
    nrows, ncols = puzzle.shape
    i, j = slot
    # if right neighbour is occupied
    if j < ncols - 1 and puzzle[i][j+1] >= 0:
        if not best_buddies(best_neighbours, RIGHT, part, puzzle[i][j+1]):
            return False
    # if left neighbour is occupied
    if j > 0 and puzzle[i][j-1] >= 0:
        if not best_buddies(best_neighbours, LEFT, part, puzzle[i][j-1]):
            return False
    # if top nieghbour is occupied
    if i > 0 and puzzle[i-1][j] >= 0:
        if not best_buddies(best_neighbours, UP, part, puzzle[i-1][j]):
            return False
    # if bottom neighbour is occupied
    if i < nrows -1 and puzzle[i+1][j] >= 0:
        if not best_buddies(best_neighbours, DOWN, part, puzzle[i+1][j]):
            return False
    return True

In [78]:
def average_compatibility_with_slot(puzzle, compatibility_matrix, slot, part):
    nrows, ncols = puzzle.shape
    i, j = slot
    total_compatibility = 0
    num_neighbours = 0
    # if right neighbour is occupied
    if j < ncols - 1 and puzzle[i][j+1] >= 0:
        total_compatibility += compatibility_matrix[RIGHT][part][puzzle[i][j+1]]
        num_neighbours += 1
    # if left neighbour is occupied
    if j > 0 and puzzle[i][j-1] >= 0:
        total_compatibility += compatibility_matrix[LEFT][part][puzzle[i][j-1]]
        num_neighbours += 1
    # if top nieghbour is occupied
    if i > 0 and puzzle[i-1][j] >= 0:
        total_compatibility += compatibility_matrix[UP][part][puzzle[i-1][j]]
        num_neighbours += 1
    # if bottom neighbour is occupied
    if i < nrows -1 and puzzle[i+1][j] >= 0:
        total_compatibility += compatibility_matrix[DOWN][part][puzzle[i+1][j]]
        num_neighbours += 1
    return total_compatibility / num_neighbours

In [93]:
def place_remaining_parts(puzzle, compatibility_matrix, unallocated_parts):
    best_neighbours = calculate_best_neighbours(compatibility_matrix)
    num_occupied_neighbours = initialise_occupied_neighbours(puzzle)
    candidate_slots = list(find_candidate_slots(num_occupied_neighbours))
    matches = [(slot, part) for slot in candidate_slots for part in unallocated_parts if does_part_fit_in_slot(puzzle, best_neighbours, slot, part)]
    if len(matches) == 1:
        slot, part = matches.pop()
    else:
        average_compatibilities = [(average_compatibility_with_slot(puzzle, compatibility_matrix, slot, part), (slot, part))
                                  for slot in candidate_slots for part in unallocated_parts]
        best = max(average_compatibilities, key=lambda x: x[0])
        slot, part = best[1]
    i, j = slot
    # update occupied neighbours
    # occupied_neighbours[i][j] = 0
    # update unallocated parts
    unallocated_parts.remove(part)
    # update puzzle
    puzzle[i][j] = part
    return puzzle, unallocated_parts

            

In [97]:
nrows = 8
ncols = 8
unallocated_parts = set(range(nrows * ncols))
best_estimated_seed = find_best_estimated_seed(best_neighbours)
puzzle = np.full((nrows, ncols), -1)
puzzle[nrows//2][ncols//2] = best_estimated_seed
unallocated_parts.remove(best_estimated_seed)
print(puzzle)

[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  0 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]


In [99]:
steps = 6
for i in range(steps):
    puzzle, unallocated_parts = place_remaining_parts(puzzle, compatibility_matrix, unallocated_parts)
    print(puzzle)

[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  0 -1 -1 -1]
 [-1 -1 -1 -1 27 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 18  0 -1 -1 -1]
 [-1 -1 -1 -1 27 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 18  0 -1 -1 -1]
 [-1 -1 -1 63 27 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 44 18  0 -1 -1 -1]
 [-1 -1 -1 63 27 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]
[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 44 18  0 -1 -1 -